In [1]:
from models.unet import Unet
from models.diffusion_model import DiffusionModel
from utils import load_config_from_file
import torch
import copy

In [ ]:
# Loading the configurations
checkpoint = torch.load('checkpoints/checkpoint_1.pth.tar')
config_unet = checkpoint('config_unet')
config_diffusion_model = checkpoint('config_diffusion_model')

# create instance of unet
unet_model = Unet(**config_unet)
unet_model.load_state_dict(checkpoint["unet_state_dict"])

ema_model = (copy.deepcopy(unet_model).eval().requires_grad_(False))
ema_model.load_state_dict(checkpoint["ema_model"])

# create instance of DiffusionModel
# keine learnable params
diffusion_model = DiffusionModel(unet_model, **config_diffusion_model)

n_samples = 10
classes = torch.arange(0, 10).to("cpu") 
samples = diffusion_model.sample(n_samples=n_samples, ema_model=None, classes=classes, cond_weight=2)
samples_ema = diffusion_model.sample(n_samples=n_samples, ema_model=ema_model, classes=classes, cond_weight=2)